In [1]:
import numpy as np
import sympy as sp
from itertools import permutations
from scipy.spatial.transform import Rotation as R
import scipy

In [2]:
class sensor:
    def recover_homogenous_affine_transformation(p, p_prime):
        '''
        Source: https://stackoverflow.com/questions/27546081/
        '''
        # construct intermediate matrix
        Q       = p[1:]       - p[0]
        Q_prime = p_prime[1:] - p_prime[0]

        # calculate rotation matrix
        R = np.dot(np.linalg.inv(np.row_stack((Q, np.cross(*Q)))),
                   np.row_stack((Q_prime, np.cross(*Q_prime))))

        # calculate translation vector
        t = p_prime[0] - np.dot(p[0], R)

        # calculate affine transformation matrix
        return np.column_stack((np.row_stack((R, t)),
                                (0, 0, 0, 1)))
    
    def __init__(self, points):
        self.trans_mat = None
        self.points = points
        self.true_coords = None
        self.beacons = []
        self.checked_dist = False
        self.overlapping_sensors = []
        for row in range(points.shape[0]):
            self.beacons.append(beacon(points[row,:]))
    
    def find_beacon_neighbors(self):
        if self.checked_dist:
            return None
        self.distances = set()
        for b in self.beacons:
            self.distances |= b.find_neighbors(self)
        self.checked_dist = True
    
    def check_overlap(self, other):
        '''if overlap is found, return T such that other.pairs @ t gives the perspective from self
        example output:
        matrix([[    0,     1,     0,     0],
                [    1,     0,     0,     0],
                [    0,     0,    -1,     0],
                [  147,   155, -1150,     1]])
        '''
        max_overlap = 0
        pairs = np.zeros((1,6))
        for b in self.beacons:
            for ob in other.beacons:
                overlapping_dist = set(b.dist.values()) & set(ob.dist.values())
                if len(overlapping_dist) >= 12:
                    pairs = np.vstack((pairs, np.hstack((b.v.reshape(1,3), ob.v.reshape(1,3)))))
                    max_overlap = np.max((max_overlap, len(overlapping_dist)))
        if max_overlap < 12:
            return False
        pairs = pairs[1:,:]
        a = pairs[:,:3]
        a = np.hstack((a, np.ones((a.shape[0],1))))
        b = pairs[:,3:]
        b = np.hstack((b, np.ones((b.shape[0],1))))
        trans_mat = sensor.recover_homogenous_affine_transformation(b[:3,:3], a[:3,:3]) #pog
        trans_mat = np.round(trans_mat).astype(int)
        return trans_mat
        
    def manhattan_distance(self, other):
        return np.sum(np.abs(self.true_coords - other.true_coords))
    def __str__(self):
        return str(self.points)
    
class beacon:
    def distance(v1, v2):
        dist = np.abs(v1-v2)
        return np.sum(np.square(dist)), np.min(dist), np.max(dist)
    
    def __init__(self, v):
        self.v = v
        self.dist = dict()
        
    def find_neighbors(self, sensor):
        for b in sensor.beacons:
            self.dist[b] = beacon.distance(self.v, b.v)
        return set(self.dist.values())


In [7]:
with open("input.txt") as f:
    data = [np.matrix(";".join(x.split('\n')[1:])) for x in f.read().split('\n\n')]
sensors = [sensor(d) for d in data]
[d.find_beacon_neighbors() for d in sensors]
print(':)')

:)


In [4]:
sen = sensors.copy() #needed for part 2, makes a new list with pointers to the original sensors
#part 1
main_sensor = sen.pop(0)
main_sensor.true_coords = np.array([0, 0, 0])
while sen:
    nxt = sen.pop(0)
    overlap = main_sensor.check_overlap(nxt)
    if overlap is False:
        sen.append(nxt)
        continue
    nxt.true_coords = overlap[3, :3].flatten()
    homo_nxt = np.hstack((nxt.points, np.ones((nxt.points.shape[0],1))))
    new_points = np.unique(np.vstack((homo_nxt@overlap[:,:3], main_sensor.points)),axis=0).astype(np.int32)
    main_sensor = sensor(new_points)
    main_sensor.find_beacon_neighbors()
print(main_sensor.points.shape)

(483, 3)


In [5]:
#part 2:
max_d = -1
for sen1 in sensors: 
    for sen2 in sensors:
        max_d = max(max_d, sen1.manhattan_distance(sen2))
print(max_d)

14804
